In [1]:
import os
os.chdir('../..')
!ls

product_impacts


In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
from skimpy import skim
from tqdm.notebook import tqdm

from product_impacts.product_cat.run_model import get_store

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tqdm.pandas()

#### Food db data

In [3]:
fdb = pd.read_csv('../../SFS/NDNS UK/predictions/all_predictions_15Oct2024.csv')

products1 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2019/foodDB_Raw/products.csv', usecols= [
    'product_id', 'product_list_name', 'product_name', 'ingredients_text',  
    'energy_per_100', 'fat_per_100', 'saturates_per_100', 'salt_per_100', 
    'sugar_per_100', 'carbohydrate_per_100', 'protein_per_100', 'fibre_per_100',
    'serving', 'serving_value', 'serving_unit']).rename(columns={
    'serving': 'serving_size', 
    'serving_value': 'serving_size_value', 
    'serving_unit': 'serving_size_unit'})
products2 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2021/foodDB_dat/products.csv', usecols= [
    'product_id', 'product_list_name', 'product_name', 'ingredients_text', 
    'energy_per_100', 'fat_per_100', 'saturates_per_100', 'salt_per_100', 
    'sugar_per_100', 'carbohydrate_per_100', 'protein_per_100', 'fibre_per_100',
    'serving', 'serving_value', 'serving_unit']).rename(columns={
    'serving': 'serving_size', 
    'serving_value': 'serving_size_value', 
    'serving_unit': 'serving_size_unit'})
products3 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2022/May_2022_Extract/products.csv',  usecols= [
    'foodDB_product_id', 'product_list_name', 'product_name', 'ingredients_list', 
    'Energy', 'Fat', 'Saturates', 'Salt', 'Sugar', 'Carbohydrate', 'Fibre', 'Protein',
    'serving_size', 'serving_size_value', 'serving_size_unit']).rename(columns={
    'foodDB_product_id': 'product_id',
    'ingredients_list': 'ingredients_text',
    'Energy': 'energy_per_100',
    'Fat': 'fat_per_100', 
    'Saturates': 'saturates_per_100', 
    'Salt': 'salt_per_100', 
    'Sugar': 'sugar_per_100', 
    'Carbohydrate': 'carbohydrate_per_100', 
    'Protein': 'protein_per_100', 
    'Fibre': 'fibre_per_100'})
products = pd.concat([products1, products2, products3], axis=0, ignore_index=True)

products = products.drop_duplicates(['product_id']).reset_index(drop=True)
products.loc[(products['product_name'].notnull()) 
             & (products['product_name'].str.len()>products['product_list_name'].str.len()),
             'product_list_name'] = products[
    (products['product_name'].notnull()) 
    & (products['product_name'].str.len()>products['product_list_name'].str.len())]['product_name']
products['ingredients_text'] = products['ingredients_text'].fillna('')

fdb = products.merge(fdb[['product_id', 'store', 'parentcategory_pred', 'mainfoodgroup_pred', 'subfoodgroup_pred']])

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (37,38,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
fdb = fdb[~fdb['parentcategory_pred'].isin(['Alcoholic Beverages', 'Not Food', 'Artificial Sweeteners'])]

In [5]:
# replacing grams with % (doing this separately here as the cleaning function function removes *)
exclusion_pattern = r'(?!.*egg\*\*)(?!.*veg\*\*)(?!.*ing\*\*)'
pattern = rf'^{exclusion_pattern}.*g\*\*.*$'
combined_condition = fdb['ingredients_text'].str.contains(pattern, case=False, regex=True)
fdb.loc[combined_condition, 'ingredients_text'] = fdb.loc[combined_condition, 
                                                          'ingredients_text'].str.replace('g**', '%', regex=False)

In [6]:
fdb.shape

(242071, 19)

In [7]:
fdb['parentcategory_pred'].value_counts()

Cereals and Cereal Products             52426
Miscellaneous                           27598
Meat and Meat Products                  26837
Non-Alcoholic Beverages                 25994
Sugar, Preserves and Confectionery      25822
Vegetables, Potatoes                    23757
Milk and Milk Products                  23239
Fish and Fish Dishes                     9302
Fruit                                    7343
Savoury Snacks                           6460
Nuts and Seeds                           5057
Fat Spreads                              3811
Commercial Toddlers Foods and Drinks     2488
Eggs and Egg Dishes                      1315
Dietary Supplements                       622
Name: parentcategory_pred, dtype: int64

#### Open food facts data

In [3]:
# it is okay to just go with the neaural network oredictions and impose a 0.8 threshold (maybe this is too conservative, can rethink)
off = pd.read_csv('../../SFS/openfoodfacts/all/openfoodfacts_lang.csv')
# off_non_eng1 = pd.read_csv('../../SFS/openfoodfacts/all/openfoodfacts_non_eng1_temp.csv')

# off_whole = off_whole[~off_whole['product_id'].isin(off_non_eng1['product_id'].unique())]
# off = pd.concat([off_whole, off_non_eng1], axis=0, ignore_index=True)

off = off[['product_id', 'product_name', 'ingredients_text',
           'product_name_en', 'ingredients_text_en', 'countries_en', 'stores',
           'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 
           'fiber_100g', 'proteins_100g', 'sugars_100g', 'salt_100g', 
           'serving_size', 'serving_quantity']].rename(columns={
    'energy_100g': 'energy_per_100', 
    'fat_100g': 'fat_per_100', 
    'saturated-fat_100g': 'saturates_per_100', 
    'salt_100g': 'salt_per_100', 
    'sugars_100g': 'sugar_per_100',
    'carbohydrates_100g': 'carbohydrate_per_100', 
    'proteins_100g': 'protein_per_100', 
    'fiber_100g': 'fibre_per_100'})

preds = []

for prefix in ['eng/', 'non_eng/']:
    pred = pd.read_csv(f'../../SFS/openfoodfacts/all/predictions/{prefix}predictions_nn.csv')

    pred = pred[(pred['parentcategory_prob']>0.8)
                      & ((pred['subfoodgroup_prob'].isna()) | (pred['subfoodgroup_prob']>0.8))]
    
    preds.append(pred)

preds = pd.concat(preds, axis=0, ignore_index=True)
off = off.merge(preds)#[['product_id', 'parentcategory', 'mainfoodgroup', 'subfoodgroup']])

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15,16,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
off = off[~off['parentcategory'].isin(['Alcoholic Beverages', 'Not Food', 'Artificial Sweeteners'])]

In [16]:
off['ingredients_text_en'] = off['ingredients_text_en'].fillna('')

In [17]:
off.shape

(851860, 22)

In [18]:
off[off['ingredients_text_en']!=''].shape

(668994, 22)

In [19]:
off[off['ingredients_text_en']!='']['countries_en'].value_counts()

United States           225155
France                  188847
Germany                  49598
United Kingdom           27809
Spain                    24811
Switzerland              17127
Italy                    16106
Belgium                  13718
World                    13400
Canada                    9243
Netherlands               8275
Australia                 6758
Poland                    5000
Ireland                   4324
Austria                   3413
Mexico                    3366
Bulgaria                  3342
Portugal                  3295
Sweden                    2990
Czech Republic            2606
Norway                    2576
New Zealand               2404
Romania                   2375
Finland                   2323
Argentina                 1888
Russia                    1767
Thailand                  1764
Bolivia                   1734
Brazil                    1499
India                     1459
Hungary                   1309
Croatia                   1217
Luxembou

In [20]:
off['countries_en'].value_counts()

United States           287081
France                  210206
Germany                  55663
United Kingdom           46711
Spain                    29344
World                    24736
Canada                   19921
Switzerland              19894
Italy                    19420
Belgium                  16507
Australia                16488
Ireland                  12260
Netherlands               9276
Poland                    5486
New Zealand               4017
Austria                   3820
Sweden                    3684
Mexico                    3613
Bulgaria                  3550
Portugal                  3481
Norway                    2993
Czech Republic            2972
Finland                   2815
Romania                   2733
India                     2202
Thailand                  2058
Brazil                    2050
Argentina                 1993
Russia                    1893
South Africa              1841
Bolivia                   1775
Singapore                 1648
Luxembou

In [21]:
off['parentcategory'].value_counts()

Cereals and Cereal Products             165729
Sugar, Preserves and Confectionery      116424
Milk and Milk Products                  102494
Meat and Meat Products                   89076
Miscellaneous                            86032
Vegetables, Potatoes                     78080
Non-Alcoholic Beverages                  71801
Nuts and Seeds                           33708
Fruit                                    31383
Fish and Fish Dishes                     30760
Savoury Snacks                           24131
Fat Spreads                              14336
Eggs and Egg Dishes                       4308
Dietary Supplements                       2750
Commercial Toddlers Foods and Drinks       848
Name: parentcategory, dtype: int64

In [22]:
off[off['countries_en'].isin(['United States', 'France', 'United Kingdom', 'Germany',
                              'Spain', 'Switzerland', 'Italy', 'Canada', 'Belgium',
                              'Australia', 'Ireland'])].groupby(
    ['countries_en', 'parentcategory']).count()[['product_id']].reset_index().pivot(
    index='countries_en', columns='parentcategory', values='product_id')

parentcategory,Cereals and Cereal Products,Commercial Toddlers Foods and Drinks,Dietary Supplements,Eggs and Egg Dishes,Fat Spreads,Fish and Fish Dishes,Fruit,Meat and Meat Products,Milk and Milk Products,Miscellaneous,Non-Alcoholic Beverages,Nuts and Seeds,Savoury Snacks,"Sugar, Preserves and Confectionery","Vegetables, Potatoes"
countries_en,,,,,,,,,,,,,,,
Australia,2584,9,40,102,229,544,756,2059,1965,1666,1348,813,493,2177,1703
Belgium,2975,14,45,87,315,652,663,1776,2020,1435,1614,639,305,2292,1675
Canada,3459,11,101,101,292,523,873,1833,2062,2027,1756,982,755,3037,2109
France,37657,321,493,1556,4416,11481,8325,27146,25275,16168,18063,5989,2628,32378,18310
Germany,9435,46,277,259,1018,1583,2094,5898,7546,5801,5010,2121,873,7307,6395
Ireland,1834,4,31,83,121,442,451,2598,1415,1049,737,537,327,1295,1336
Italy,5260,22,42,76,317,913,606,1213,3057,1218,1545,654,283,2269,1945
Spain,5335,12,124,119,641,1210,1104,2216,4201,2304,2565,914,544,4660,3395
Switzerland,3648,30,73,102,389,644,844,1679,2879,1775,1871,672,263,3199,1826


#### Generic cleaning funciton - can be applied to both food db and open food facts

In [17]:
def clean_ingredients(text, is_fdb=True):
    
    # Mapping for special characters not handled by unicodedata
    special_replacements = {
        'ß': 'ss',
        'Æ': 'ae',
        'æ': 'ae',
        'Œ': 'oe',
        'œ': 'oe',
        'Ø': 'o',
        'ø': 'o',
        'Đ': 'd',
        'đ': 'd',
        'Þ': 'th',
        'þ': 'th',
        # Add more mappings as needed
    }
    
    # Removing unwanted characters and words
    text = text.lower()
    text = text.replace("*", " ").replace("_", " ").replace("’", "'").replace("?", " ").replace("=", " ").replace("^", " ")
    text = re.sub(' +', ' ', text)
    text = text.replace(";", ",").replace(",,", ",").replace(", ,", ",")
    text = text.replace(": (", "(")
    text = text.replace("“", "").replace("”", "")
    text = text.replace("& quot;", "")  # remove html encoded quotes
    text = text.replace("â€\xa0", "")
    text = text.replace("\u200b", "")
    text = text.replace("&", "and")      # replace ampersand
    text = text.replace('ingredients:', '')
    text = text.replace('ingredients list:', '')
    text = re.sub(r'(?<!\s)\(', ' (', text)
    
    # removing diacritics and replacing special Latin characters
    for char, replacement in special_replacements.items():
        text = text.replace(char, replacement)
    # Normalize the string to NFD (Normalization Form Decomposition)
    text = unicodedata.normalize('NFD', text)
    # Remove combining diacritical marks
    text = re.sub(r'[\u0300-\u036f]', '', text)
    
    # Converting brackets to parentheses
    text = re.sub("\[|\{","(", text)
    text = re.sub("\]|\}",")", text)

    # Replacing "per cent" with "%"
    text = re.sub(r"per(\s)?cent", "%", text)
    text = text.replace(" %", "%")
    text = text.replace("%", "% ")
    text = text.replace("( ", "(").replace(" )", ")")
    text = re.sub(' +', ' ', text)
    text = text.replace("%%", "%")
    
    # Fix "(10% brown" → "(10%) brown"
    text = re.sub(r'\((\d+(?:\.\d+)?)%(\s)', r'(\1%)\2', text)
    # Fix "grapes 10%)" → "grapes (10%)" 
    text = re.sub(r'\s+([+-]?\d+(?:\.\d+)?)%\)', r' (\1%)', text)
    # Fix "10%" → "(10%)"
    text = re.sub(r'(?<!\()(\d+(?:\.\d+)?)%(?!\))', r'(\1%)', text)
    
    # Removing specific allergen advice text and other labels
    if is_fdb:
        text = text.replace("allergy Advice: for allergens see highlighted ingredients", "")
        text = text.replace("for allergens, see undelined ingredients in bold.", "")
    text = re.sub(r'allerg.*', '', text)
    text = re.sub(r'it may contain.*', '', text)
    text = re.sub(r'may contain.*', '', text)
    text = re.sub(r'our method.*', '', text)
    text = re.sub(r'www.*', '', text)
    text = re.sub(r'dietary advice.*', '', text)
    text = re.sub(r'not suitable for.*', '', text)
    text = re.sub(r'suitable for.*', '', text)
    text = re.sub(r'this product is made in.*', '', text)
    text = re.sub(r'made in.*', '', text)
    text = re.sub(r'free from.*', '', text)
    text = re.sub(r'rainforest alliance.*', '', text)
    text = re.sub(r'organic ingredients certified.*', '', text)
    text = re.sub(r'certified organic ingredients.*', '', text)
    text = re.sub(r'from sustainably grown.*', '', text)
    text = text.replace('sustainably grown', '')
    text = re.sub(r'†from.*', '', text)
    text = text.replace("†", " ").strip()
    text = re.sub(' +', ' ', text)

    # removing empty brackets
    text = text.replace("()"," ").strip()
    text = re.sub(' +', ' ', text)
    
    return text

In [18]:
fdb['ingredients_text'] = fdb.progress_apply(lambda row: clean_ingredients(row['ingredients_text'], True), axis=1)

  0%|          | 0/242071 [00:00<?, ?it/s]

In [19]:
off['ingredients_text_en'] = off.progress_apply(lambda row: clean_ingredients(row['ingredients_text_en'], False), axis=1)

  0%|          | 0/851860 [00:00<?, ?it/s]

#### cleaning functions only for open food facts

In [20]:
def clean_ingredients2(text):
    
    # handling ' - ' separaters 
    if ',' not in text:
        text = text.replace(" - ", ", ")
        
    # removing origin: ...
    text = re.sub("\(origin:[^)]*\)", "", text)
    text = re.sub("\(country of origin:[^)]*\)", "", text)
    text = re.sub("origin:[^.]*\.", "", text)
    
    text = re.sub(r'although every care.*', '', text)
    
    return text

In [21]:
off['ingredients_text_en'] = off.apply(lambda row: clean_ingredients2(row['ingredients_text_en']), axis=1)

In [22]:
off = off[off['countries_en'].isin(['United States', 'France', 'United Kingdom', 'Germany',
                                    'Spain', 'Switzerland', 'Italy', 'Canada', 'Belgium',
                                    'Australia', 'Ireland'])].reset_index(drop=True)

In [23]:
# 2 rows have large and weird strings, removing those
off.loc[(off['ingredients_text_en'].str.contains('helado sabor a vainilla 55,(5%)', regex=False)), 
        'ingredients_text_en'] = ''

In [24]:
allowed_pattern = r'^[A-Za-z0-9.\(\),% ]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas
off1 = off[(off['ingredients_text_en'].str.match(allowed_pattern))
           | (off['ingredients_text_en']=='')
          ]

off2 = off[(~off['ingredients_text_en'].str.match(allowed_pattern))
           & (off['ingredients_text_en']!='')
          ]

In [25]:
off1.shape

(452455, 22)

In [26]:
off2.shape # figure out if any of these are usable

(281040, 22)

In [27]:
off1['countries_en'].value_counts()

United States     193036
France            104344
United Kingdom     34568
Germany            30048
Spain              18172
Canada             15868
Australia          14462
Switzerland        11707
Italy              10838
Ireland            10057
Belgium             9355
Name: countries_en, dtype: int64

In [28]:
off1[off1['ingredients_text_en']!='']['countries_en'].value_counts()

United States     131074
France             82890
Germany            23969
United Kingdom     15630
Spain              13629
Switzerland         8930
Italy               7517
Belgium             6558
Canada              5188
Australia           4725
Ireland             2114
Name: countries_en, dtype: int64

In [29]:
off2['countries_en'].value_counts()

France            105862
United States      94045
Germany            25615
United Kingdom     12143
Spain              11172
Italy               8582
Switzerland         8187
Belgium             7152
Canada              4053
Ireland             2203
Australia           2026
Name: countries_en, dtype: int64

In [30]:
def correct_structure(s, max_iterations=10):
    """
    Corrects the structure of the input string by ensuring that percentages
    are placed immediately after the relevant words within each segment.

    Args:
        s (str): The input string to be corrected.

    Returns:
        str: The corrected string with properly structured segments.
    """
    s = re.sub(' +', ' ', s)
    s = re.sub(r'\.$', '', s)
    s = re.sub(r'\,$', '', s)

    # Pattern 1: (percentage) word => word (percentage)
    pattern1 = re.compile(r'\((\d+(\.\d+)?)%\)\s+([^(),]+)')
    replacement1 = r'\3 (\1%)'

    # Pattern 2: word (description) (percentage) => word (percentage) (description)
    pattern2 = re.compile(r'([^(),]+)\s*\(([^()]+)\)\s*\((\d+(\.\d+)?)%\)')
    replacement2 = r'\1 (\3%) (\2)'

    iterations = 0
    previous_s = None

    while previous_s != s and iterations < max_iterations:
        previous_s = s
        s = pattern1.sub(replacement1, s)
        s = pattern2.sub(replacement2, s)
        iterations += 1

    if iterations >= max_iterations:
        # Optionally, log the problematic string for debugging
        # print(f"Max iterations reached. Replacing string with empty string: {s}")
        return ''
    
    # Clean up any extra spaces resulting from substitutions
    s = re.sub(r'\s+', ' ', s)
    
    s = s.replace('(', ' (')
    s = re.sub(' +', ' ', s)

    return s.strip()

In [31]:
off1['ingredients_text_en'] = off1['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/452455 [00:00<?, ?it/s]

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
off1[off1['ingredients_text_en']!='']['countries_en'].value_counts()

United States     130909
France             81897
Germany            23880
United Kingdom     15521
Spain              13430
Switzerland         8874
Italy               7422
Belgium             6502
Canada              5173
Australia           4667
Ireland             2100
Name: countries_en, dtype: int64

In [33]:
def clean_ingredients3(text):
    # remove may contain, traces etc
    
    text = re.sub(r'it can contain.*', '', text)
    text = re.sub(r'can contain.*', '', text)
    text = re.sub(r'it does not contain.*', '', text)
    text = re.sub(r'this product coes not contain.*', '', text)
    text = re.sub(r'this chocolate coes not contain.*', '', text)
    text = re.sub(r'does not contain.*', '', text)
    text = re.sub(r'possible traces.*', '', text)
    text = re.sub(r'keep refrigerated.*', '', text)
    
    text = re.sub(r'contains or less of the following:', '', text)
    text = re.sub(r'contains less than of the following:', '', text)
    text = re.sub(r'contains two% or less of the following:', '', text)
    
    text = re.sub(r'contains or less of :', '', text)
    text = re.sub(r'contains less than of :', '', text)
    text = re.sub(r'contains two% or less of :', '', text)
    
    text = re.sub(r'contains or less of the following', '', text)
    text = re.sub(r'contains less than of the following', '', text)
    text = re.sub(r'contains two% or less of the following', '', text)
    
    text = re.sub(r'contains or less of', '', text)
    text = re.sub(r'contains less than of', '', text)
    text = re.sub(r'contains two% or less of', '', text)
    
    text = re.sub(r'contains or less', '', text)
    text = re.sub(r'contains less than', '', text)
    text = re.sub(r'contains two% or less', '', text)
    
    text = re.sub(r'\.$', '', text)
    text = re.sub(r'\,$', '', text)
    text = re.sub(r'(?<!\d)\.', ',', text)
    
    return text.strip()

In [34]:
off1['ingredients_text_en'] = off1['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/452455 [00:00<?, ?it/s]

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
off1.shape

(452455, 22)

In [36]:
off1[off['ingredients_text_en']!=''].shape

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(302224, 22)

In [37]:
off1[off['ingredients_text_en']!='']['countries_en'].value_counts()

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


United States     131074
France             82890
Germany            23969
United Kingdom     15630
Spain              13629
Switzerland         8930
Italy               7517
Belgium             6558
Canada              5188
Australia           4725
Ireland             2114
Name: countries_en, dtype: int64

In [38]:
allowed_pattern = r'^[A-Za-z0-9.\(\),% -]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas, hyphen
off3 = off2[(off2['ingredients_text_en'].str.match(allowed_pattern))]

off4 = off2[(~off2['ingredients_text_en'].str.match(allowed_pattern))]

In [39]:
off2.shape

(281040, 22)

In [40]:
off3.shape

(45750, 22)

In [41]:
off4.shape

(235290, 22)

In [42]:
off3 = off3[(off3['ingredients_text_en'].str.contains('-', regex=False)) 
            & (~off3['ingredients_text_en'].str.contains(' - ', regex=False))]
off3.shape

(35040, 22)

In [43]:
off3['ingredients_text_en'] = off3['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/35040 [00:00<?, ?it/s]

In [44]:
off3['ingredients_text_en'] = off3['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/35040 [00:00<?, ?it/s]

In [45]:
allowed_pattern = r'^[A-Za-z0-9.\(\),%: ]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas, colon
off5 = off4[(off4['ingredients_text_en'].str.match(allowed_pattern))]

off6 = off4[(~off4['ingredients_text_en'].str.match(allowed_pattern))]

In [46]:
off5.shape

(112462, 22)

In [47]:
off6.shape

(122828, 22)

In [48]:
off5 = off5[(off5['ingredients_text_en'].str.contains('preservative:', regex=False))
             | (off5['ingredients_text_en'].str.contains('preservatives:', regex=False))
             | (off5['ingredients_text_en'].str.contains('conservative:', regex=False))
             | (off5['ingredients_text_en'].str.contains('conservatives:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidifying:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidity:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidic:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acid:', regex=False))
             | (off5['ingredients_text_en'].str.contains('color:', regex=False))
             | (off5['ingredients_text_en'].str.contains('colors:', regex=False))
             | (off5['ingredients_text_en'].str.contains('colour:', regex=False))
             | (off5['ingredients_text_en'].str.contains('coloring:', regex=False))
             | (off5['ingredients_text_en'].str.contains('regulator:', regex=False))
             | (off5['ingredients_text_en'].str.contains('regulators:', regex=False))
             | (off5['ingredients_text_en'].str.contains('corrector:', regex=False))
             | (off5['ingredients_text_en'].str.contains('emulgator:', regex=False))
             | (off5['ingredients_text_en'].str.contains('emulsifying:', regex=False))
             | (off5['ingredients_text_en'].str.contains('antioxidant:', regex=False))
             | (off5['ingredients_text_en'].str.contains('fier:', regex=False))
             | (off5['ingredients_text_en'].str.contains('fiers:', regex=False))
             | (off5['ingredients_text_en'].str.contains('zer:', regex=False))
             | (off5['ingredients_text_en'].str.contains('less:', regex=False))
             | (off5['ingredients_text_en'].str.contains('following:', regex=False))
             | (off5['ingredients_text_en'].str.contains('of:', regex=False))
             | (off5['ingredients_text_en'].str.contains('thickening:', regex=False))
             | (off5['ingredients_text_en'].str.contains('contains:', regex=False))
             | (off5['ingredients_text_en'].str.contains('cultures:', regex=False))
             | (off5['ingredients_text_en'].str.contains('agent:', regex=False))]

In [49]:
off5['ingredients_text_en'] = off5['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/82238 [00:00<?, ?it/s]

In [50]:
off5['ingredients_text_en'] = off5['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/82238 [00:00<?, ?it/s]

In [51]:
off_clean = pd.concat([off1, off3, off5], axis=0, ignore_index=True)

In [52]:
def clean_ingredients4(text):
    # remove may contain, traces etc
    
    text = re.sub(r'product from organic farming.*', '', text)
    text = re.sub(r'products from organic farming.*', '', text)
    text = re.sub(r'all the ingredients come from organic farmin.*', '', text)
    text = re.sub(r'agricultural ingredient from organic farming.*', '', text)
    text = re.sub(r'(ingredients from organic farming).*', '', text)
    text = re.sub(r'ingredient from organic farming.*', '', text)
    text = re.sub(r'ingredients from organic farming.*', '', text)
    text = re.sub(r'from organic farming.*', '', text)
    
    text = text.replace('origin france', '')
    text = text.replace('origin, france', '')
    text = text.replace('developed from eu origin', '')
    text = text.replace('eu origin, asia', '')
    text = text.replace('eu original', '')
    text = text.replace('eu origin', '')
    text = text.replace('origin eu', '')
    text = text.replace('origin spain', '')
    text = text.replace('brittany origin', '')
    text = text.replace('origin of the netherlands and france', '')
    text = text.replace('zygochlams patagonjca, argentinian origin', '')
    text = text.replace('ingredients origin france (100%)', '')
    text = text.replace('developed from france origin france', '')
    text = text.replace('this product is made from eu original pork', '')
    text = text.replace('origin england or france', '')
    text = text.replace('ingredients of natural origin', '')
    text = text.replace('france', '')
    
    text = text.replace(':', '')
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text = re.sub(r'\.$', '', text)
    text = re.sub(r'\,$', '', text)
    text = text.replace('()', '')
    
    return text.strip()

In [53]:
off_clean['ingredients_text_en'] = off_clean['ingredients_text_en'].progress_apply(clean_ingredients4)

  0%|          | 0/569733 [00:00<?, ?it/s]

In [54]:
off_clean.shape

(569733, 22)

In [55]:
off_clean['countries_en'].value_counts()

United States     230217
France            149933
Germany            41327
United Kingdom     39967
Spain              22493
Canada             17151
Switzerland        15692
Australia          15193
Italy              14414
Belgium            12265
Ireland            11081
Name: countries_en, dtype: int64

In [56]:
off_clean[off_clean['ingredients_text_en']!=''].shape

(415770, 22)

In [57]:
off_clean[off_clean['ingredients_text_en']!='']['countries_en'].value_counts()

United States     167932
France            126237
Germany            35051
United Kingdom     20863
Spain              17674
Switzerland        12800
Italy              10907
Belgium             9366
Canada              6442
Australia           5383
Ireland             3115
Name: countries_en, dtype: int64

In [58]:
off_clean.groupby(
    ['countries_en', 'parentcategory']).count()[['product_id']].reset_index().pivot(
    index='countries_en', columns='parentcategory', values='product_id')

parentcategory,Cereals and Cereal Products,Commercial Toddlers Foods and Drinks,Dietary Supplements,Eggs and Egg Dishes,Fat Spreads,Fish and Fish Dishes,Fruit,Meat and Meat Products,Milk and Milk Products,Miscellaneous,Non-Alcoholic Beverages,Nuts and Seeds,Savoury Snacks,"Sugar, Preserves and Confectionery","Vegetables, Potatoes"
countries_en,,,,,,,,,,,,,,,
Australia,2326,9,29,102,216,514,744,1935,1786,1503,1265,789,438,1922,1615
Belgium,2056,12,25,66,256,465,592,1169,1298,1121,1325,599,254,1649,1378
Canada,2678,11,49,90,255,482,853,1659,1758,1736,1554,927,603,2575,1921
France,23790,219,246,1277,3628,8017,6893,19587,15380,12233,14694,5214,2097,22035,14623
Germany,6536,37,155,223,903,1174,1820,4119,5035,4423,4113,1896,679,5093,5121
Ireland,1553,4,22,74,112,409,436,2446,1229,926,684,520,299,1129,1238
Italy,3444,18,19,73,275,726,522,933,2038,926,1332,609,251,1605,1643
Spain,4009,10,64,107,580,980,914,1805,2632,1887,2154,846,445,3301,2759
Switzerland,2619,24,36,90,339,537,729,1311,2138,1461,1575,611,226,2415,1581


In [59]:
# off_clean[off_clean['ingredients_text_en']!=''].sample(100)['ingredients_text_en'].values

In [60]:
off[~off['product_id'].isin(off_clean['product_id'].unique())].shape

(163762, 22)

In [61]:
# off[~off['product_id'].isin(off_clean['product_id'].unique())].sample(100)['ingredients_text_en'].values

In [62]:
###########################
# maybe also try : and - together
# translate some words - ble: wheat, soja: soya, oeuf: egg, leche: milk

In [63]:
def translation_terms(text):
    
    text = text.replace(' ble ', ' wheat ')
    text = text.replace('soja ', ' soya ')
    text = text.replace(' leche ', ' milk ')
    text = text.replace(' ait ', ' milk ')
    text = text.replace(' oeuf ', ' egg ')
    text = text.replace(' euf ', ' egg ')
    text = text.replace(' oeufs ', ' eggs ')
    text = text.replace(' eufs ', ' eggs ')
    text = text.replace(' moutarde ', ' mustard ')
    text = text.replace(' sucre ', ' sugar ')
    text = text.replace(' beurre ', ' butter ')
    
    return text

In [64]:
off_clean['ingredients_text_en'] = off_clean['ingredients_text_en'].progress_apply(translation_terms)

  0%|          | 0/569733 [00:00<?, ?it/s]

#### standardize columns and merge datasets

In [65]:
# cleaning nutrient data
def nutrition_adjust_function(dat, nutrient_list):
    """
    Adjusts and standardizes nutrient data in a DataFrame.

    Parameters:
    - dat: pandas DataFrame containing nutrient information.
    - nutrient_list: list of nutrient column names to process.

    Returns:
    - Modified DataFrame with standardized nutrient values and units.
    """
    # Creating new columns to store data
    for nutrient in nutrient_list:
        dat[f"{nutrient}_value"] = np.nan
        dat[f"{nutrient}_unit"] = np.nan

    # Define units to extract
    unit_pattern = r'(mg|g|kcal|calorie|kj|j|kiloj)'

    # Define which nutrients are energy
    energy_nutrients = ['energy_per_100']

    for nutrient in nutrient_list:
        # Process each nutrient column
        value_col = f"{nutrient}_value"
        unit_col = f"{nutrient}_unit"

        # Ensure the nutrient column is string type for regex operations
        dat[nutrient] = dat[nutrient].astype(str)

        # Extract units using regex
        dat[unit_col] = dat[nutrient].str.extract(unit_pattern, flags=re.IGNORECASE, expand=False)
        dat[unit_col] = dat[unit_col].str.lower()

        # Extract numeric values using regex
        # Pattern to capture numbers with optional decimal separators and optional spaces
        number_pattern = r'(\d{1,4}\s*[.,]?\s*\d{0,3})'

        extracted_numbers = dat[nutrient].str.extract(number_pattern, flags=re.IGNORECASE, expand=False)
        extracted_numbers = extracted_numbers.str.replace(r'\s+', '', regex=True)  # Remove spaces
        extracted_numbers = extracted_numbers.str.replace(',', '.', regex=False)  # Replace comma with dot

        # Assign numeric values
        dat[value_col] = pd.to_numeric(extracted_numbers, errors='coerce')

        # Handle entries that are purely numeric (no units)
        # Detect if the entire string is a number
        purely_numeric_mask = dat[nutrient].str.strip().str.fullmatch(r'^\d+([.,]\d+)?$')

        # Assign values and units for purely numeric entries
        dat.loc[purely_numeric_mask, value_col] = pd.to_numeric(dat.loc[purely_numeric_mask, nutrient].str.replace(',', '.', regex=False), errors='coerce')
        # Assign default units: 'kj' for energy, 'g' for others
        default_unit = 'kj' if nutrient.lower() in [n.lower() for n in energy_nutrients] else 'g'
        dat.loc[purely_numeric_mask, unit_col] = default_unit

        # Adjusting units
        # Convert kcal to kj
        kcal_mask = dat[unit_col] == 'kcal'
        dat.loc[kcal_mask, value_col] = dat.loc[kcal_mask, value_col] * 4.184
        dat.loc[kcal_mask, unit_col] = 'kj'

        # Convert mg to g
        mg_mask = dat[unit_col] == 'mg'
        dat.loc[mg_mask, value_col] = dat.loc[mg_mask, value_col] / 1000
        dat.loc[mg_mask, unit_col] = 'g'

        # Define expected unit based on nutrient type
        expected_unit = 'kj' if nutrient.lower() in [n.lower() for n in energy_nutrients] else 'g'

        # Identify invalid units
        invalid_unit_mask = ~dat[unit_col].isin([expected_unit])

        # Set invalid entries to NaN
        dat.loc[invalid_unit_mask, value_col] = np.nan
        dat.loc[invalid_unit_mask, unit_col] = np.nan

        # Additionally, if the value is NaN, set unit to NaN
        value_nan_mask = dat[value_col].isna()
        dat.loc[value_nan_mask, unit_col] = np.nan

    return dat

In [66]:
# Define the list of nutrients
nutrient_list = ['energy_per_100', 'fat_per_100', 'saturates_per_100', 
                 'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
                 'sugar_per_100', 'salt_per_100']

# Apply the function
fdb = nutrition_adjust_function(fdb, nutrient_list)

In [67]:
# Apply the function
off_clean = nutrition_adjust_function(off_clean, nutrient_list)

In [68]:
off_clean['serving_size'] = off_clean['serving_size'].fillna('')
off_clean['serving_size_unit'] = 'g'
off_clean.loc[(~off_clean['serving_size'].str.contains('g'))
              & (off_clean['serving_size']!=''), 'serving_size_unit'] = 'ml'

In [69]:
off_clean = off_clean[['product_id', 'product_name_en', 'ingredients_text_en',
                       'countries_en', 'stores', 
                       'energy_per_100', 'fat_per_100', 'saturates_per_100', 
                       'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
                       'sugar_per_100', 'salt_per_100',
                       'energy_per_100_value', 'fat_per_100_value', 'saturates_per_100_value', 
                       'carbohydrate_per_100_value', 'fibre_per_100_value', 'protein_per_100_value', 
                       'sugar_per_100_value', 'salt_per_100_value',
                       'energy_per_100_unit', 'fat_per_100_unit', 'saturates_per_100_unit', 
                       'carbohydrate_per_100_unit', 'fibre_per_100_unit', 'protein_per_100_unit', 
                       'sugar_per_100_unit', 'salt_per_100_unit',
                       'serving_size', 'serving_quantity', 'serving_size_unit',
                       'parentcategory', 'mainfoodgroup','subfoodgroup']].rename(
    columns={'product_name_en': 'product_name', 
             'ingredients_text_en': 'ingredients_text',
             'countries_en': 'country',
             'stores': 'store',
             'serving_quantity': 'serving_size_value',
             'parentcategory': 'Department', 
             'mainfoodgroup': 'Aisle',
             'subfoodgroup': 'Shelf'})

In [70]:
off_clean['database'] = 'openfoodfacts'

In [71]:
off_clean.shape

(569733, 36)

In [72]:
fdb['country'] = 'United Kingdom'
fdb = fdb[['product_id', 'product_list_name', 'ingredients_text', 
           'country', 'store', 
           'energy_per_100', 'fat_per_100', 'saturates_per_100', 
           'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
           'sugar_per_100', 'salt_per_100',
           'energy_per_100_value', 'fat_per_100_value', 'saturates_per_100_value', 
           'carbohydrate_per_100_value', 'fibre_per_100_value', 'protein_per_100_value', 
           'sugar_per_100_value', 'salt_per_100_value',
           'energy_per_100_unit', 'fat_per_100_unit', 'saturates_per_100_unit', 
           'carbohydrate_per_100_unit', 'fibre_per_100_unit', 'protein_per_100_unit', 
           'sugar_per_100_unit', 'salt_per_100_unit',
           'serving_size', 'serving_size_value', 'serving_size_unit',
           'parentcategory_pred', 'mainfoodgroup_pred','subfoodgroup_pred']].rename(
    columns={'product_list_name': 'product_name',
             'parentcategory_pred': 'Department',
             'mainfoodgroup_pred': 'Aisle',
             'subfoodgroup_pred': 'Shelf'})

In [73]:
fdb.loc[(~fdb['serving_size_unit'].isin(['', 'g', 'ml']))
        & (fdb['serving_size_unit'].notnull()), 'serving_size_value'] = np.nan
fdb.loc[(~fdb['serving_size_unit'].isin(['', 'g', 'ml']))
        & (fdb['serving_size_unit'].notnull()), 'serving_size_unit'] = ''
fdb['database'] = 'fooddb'

In [74]:
fdb.shape

(242071, 36)

In [75]:
products_cats = pd.concat([off_clean, fdb], axis=0, ignore_index=True)

In [76]:
products_cats.shape

(811804, 36)

In [77]:
products_cats['country_group'] = 'UK'
products_cats.loc[products_cats['country'].isin(['France', 'Germany', 'Switzerland', 
                                                 'Belgium']), 'country_group'] = 'EUwest' 
products_cats.loc[products_cats['country'].isin(['Italy', 'Spain']), 'country_group'] = 'EUsouth' 

In [78]:
# products_cats.sample(100)

In [79]:
# can think about department aisle shelf, maybe rename columns according to R script, add more rows to off_clean with or without ingredients

In [80]:
# maybe do this before cleaning - and remove this from r script (lines 358-364), also 798 to 804

# products_cats[products_cats['ingredients_text'].str.contains(r'%.*?\b(fat|less|reduced|lower)\b', 
#                                                              case=False, regex=True)].shape

In [81]:
products_cats[products_cats['ingredients_text']!=''].shape

(627268, 37)

In [82]:
products_cats[products_cats['ingredients_text']!='']['country'].value_counts()

United Kingdom    232361
United States     167932
France            126237
Germany            35051
Spain              17674
Switzerland        12800
Italy              10907
Belgium             9366
Canada              6442
Australia           5383
Ireland             3115
Name: country, dtype: int64

In [86]:
products_cats.to_csv('../../SFS/environmental_impacts/Products_dat/products_categories.csv', index=False)